In [13]:
!pip install python-dotenv
!pip install openai

# Getting data from Mergeflow's API

In [39]:
import requests
import json
import os
from dotenv import load_dotenv # stores API keys for OpenAI and Mergeflow 

load_dotenv()  # Load variables from .env file
mergeflow_api_key = os.getenv('MERGEFLOW_API_KEY') # Assign Mergeflow API

# the API call
query = 'factory+automation' # topic is "large language model"factory automation
dataset = '&q=*%5bfundingcompany%7cfundingproject%7cfundingorganization%5d&sp=2572' # access dataset from mergeflow - in this case venture capital investments
rows = str(10) # first 10 docs only

# in the url below, replace "fw" by your user name. You can see your user name at mergeflow.net
# -> Account -> the string in angle brackets
url = 'https://mergeflow.net/api/v1/ferdinandk/search/getContent?q=' + query + dataset + '&rows=' + rows

# Define the headers to be sent with the request
headers = {
    'MergeflowNet-Api-Auth-Key': mergeflow_api_key
}

# Get and print the response
response = requests.post(url, headers=headers)

mergeflow_docs = []

# Check the response status code and content
if response.status_code == 200:
    mergeflow_response_json = response.json()
    
    # Pretty-print the JSON response -- useful to understand what the JSON looks like
    #pretty_json = json.dumps(mergeflow_response_json, indent=4)
    #print(pretty_json)
    
    # get title, URL, date, and content
    for document in mergeflow_response_json['Documents']:
        current_doc = {
            'title': document['Title'],
            'date': document['Date'],
            'url': document['Url'],
            'content': document['Content']
        }
        
        mergeflow_docs.append(current_doc)
        #print(content)        

    print('Found ' + str(len(mergeflow_docs)) + ' documents.')

else:
    print('Request failed with status code:', response.status_code)
    print(response.text)  # Print the response text


Found 10 documents.


# Answering questions on the results with OpenAI GPT

In [66]:
import openai

openai.api_key = os.getenv('OPENAI_API_KEY') # Assign OpenAI API

system_prompt = f"""You are Senior Vice President Technology and Innovation & CTO of Factory Automation. Your work is highly cruical and you are responsible of coming up with the next best innovation.
        """
problem = {}
i = 0

for doc in mergeflow_docs: #stores all the problems in 1 string 
    user_prompt = f"""
    Consider the text below, delimited by ```, in the area of {query}:
    ```
    {doc['content']}
    ```    
    You want to find pressing problems of your customers which would be worth solving.
    - Your customers are manufacturing companies across various sectors such as automotive, electronics, consumer goods, pharmaceuticals, food and beverage, and aerospace. 
    - Please find 1 specific problem which seems highly problematic for your clients.
    - For this problem, please name the specific resulting pain point of the customer and the customer journey.
    - For this problem, please specify which type of customer has the identified problem most often.
    - For this problem, please write only 75 words. 
    
    """
    
    completion = openai.chat.completions.create(
        model="gpt-3.5-turbo-0613",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0.7,
        max_tokens=2000,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
    )
    problem[i] = completion.choices[0].message
    i += 1
    
    
#Rate the problems from the 1 string and give out top 3     
user_prompt1 = f"""
    Consider the following problems {problem}:
    ```    
    Please rank the different problems according to the following criteria:
    - Amount of customers who have this problem (the more customers, the better)
    - Need to be fixed and hence the willingness to pay of these customers (the higher the willingness to pay, the better)
    - Amount of existent solutions on the market (the fewer effective existing solutions to the problem, the better)
    
    Please make sure that the ideas ranked in the first three places are not very similar to each other.
    
    Please give me more details about the 3 problems ranked highest:
    For each problem write a 50 word paragraph containing the following clearly separated 5 contents: 
    1) Start the problem with the ranking "Problem with rank X out of 10". For example, if you have 10 problems your second highest rated problem should start with "Problem with Rank 2 out of 10"
    2) State the problem
    3) Name the specific resulting pain point of the customer
    4) Specify which type of customers has the identified problem most often
    5) Short reasoning for priority (1 sentence)
    
    """
    

completion = openai.chat.completions.create(
model="gpt-3.5-turbo-0613",
messages=[
{"role": "system", "content": system_prompt},
{"role": "user", "content": user_prompt1}
],
temperature=0.7,
max_tokens=2000,
top_p=0.95,
frequency_penalty=0,
presence_penalty=0,
stop=None
)

    
print(completion.choices[0].message)


ChatCompletionMessage(content='Based on the given criteria, here is the ranking of the problems:\n\n1) Problem with Rank 1 out of 10:\n   - Problem: Inefficient production line changeovers\n   - Pain Point: Manufacturing companies in sectors like consumer goods and electronics often struggle with inefficient production line changeovers, causing significant downtime, delays, and increased costs.\n   - Most Affected Customer: Manufacturing companies in industries with frequent product changes or customization requirements.\n   - Reasoning: This problem affects a wide range of customers across different sectors, and addressing it would streamline operations, reduce downtime, and improve productivity.\n\n2) Problem with Rank 2 out of 10:\n   - Problem: Inefficient inventory management leading to production delays and increased costs\n   - Pain Point: Manufacturing companies across various sectors struggle with inaccurate inventory management, resulting in production delays and increased co